In [1]:
from flask import Flask, request, jsonify, render_template

app = Flask(__name__)

# Tax Brackets for Nigeria
def calculate_tax(income, deductions):
    CRA = max(0.2 * income + 200000, 0)  # Consolidated Relief Allowance
    taxable_income = max(income - deductions - CRA, 0)

    tax_brackets = [
        {'rate': 0.07, 'limit': 300000},
        {'rate': 0.11, 'limit': 600000},
        {'rate': 0.15, 'limit': 1100000},
        {'rate': 0.19, 'limit': 1600000},
        {'rate': 0.21, 'limit': 3200000},
        {'rate': 0.24, 'limit': float('inf')},
    ]

    total_tax = 0
    remaining_income = taxable_income

    for bracket in tax_brackets:
        if remaining_income <= 0:
            break
        taxable_at_this_rate = min(remaining_income, bracket['limit'])
        total_tax += taxable_at_this_rate * bracket['rate']
        remaining_income -= taxable_at_this_rate

    net_income = income - total_tax - deductions

    return {
        'gross_income': income,
        'total_deductions': deductions,
        'taxable_income': taxable_income,
        'total_tax': total_tax,
        'net_income': net_income
    }

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/calculate', methods=['POST'])
def calculate():
    data = request.json
    income = float(data.get('income', 0))
    deductions = float(data.get('deductions', 0))
    result = calculate_tax(income, deductions)
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
